# pandas 进阶修炼 ｜早起Python
<br>

**本习题由公众号【早起Python & 可视化图鉴】 原创，转载及其他形式合作请与我们联系（微信号`sshs321`)，未经授权严禁搬运及二次创作，侵权必究！**



本习题基于 `pandas` 版本 `1.1.3`，所有内容应当在 `Jupyter Notebook` 中执行以获得最佳效果。


不同版本之间写法可能会有少许不同，如若碰到此情况，你应该学会如何自行检索解决。

## 7 - 数据透视与合并



<br>

**<font color = '#5172F0'><font size=3.5>必读👇👇👇**</font>
    
现在让我们继续练习 pandas数据分析另一组常用操作 --> **数据透视与合并**


本节习题将涉及四大函数：
    
- pivot_table
- concat
- merge
- join

随着练习的深入，若没有一定的基础知识将很难继续刷题
    
官方文档永远是最好的学习手册，在本节之前强烈建议学习[官方文档对应部分](https://pandas.pydata.org/pandas-docs/version/0.20/merging.html#database-style-dataframe-joining-merging)
 




## 初始化

<br>

该 `Notebook` 版本为**习题+答案版**


请执行下方单元格以初始化读取本节习题的答案

执行完毕后，你可以使用 `ans(1)` 来查看第 1 题的参考解答，其余习题类似。

注意：所有答案并非固定（唯一），我提供的答案仅供参考（并非推荐答案或者最优解）

如果发现任何错误或者你有更好的解答，可以微信搜索公众号「早起Python」提交以获得奖励！


In [1]:
import sys
sys.path.append('../document/')
from inits import initialize, ans
res = initialize("../document/answer-7.txt")

***************正在初始化***************
*************读取到配置文件*************
***************初始化成功***************


## 数据透视表

![](http://liuzaoqi.oss-cn-beijing.aliyuncs.com/2021/09/14/16316101294678.jpg?域名/sample.jpg?x-oss-process=style/stylename)

### 1 - 加载数据

读取当前目录下 `"某超市销售数据.csv"` 并设置千分位符号为 `,`

In [49]:
import pandas as pd

df = pd.read_csv("某超市销售数据.csv",thousands=',')

### 2 - 数据透视｜默认

制作各省「平均销售额」的数据透视表

In [ ]:
pd.pivot_table(df,values = ['销售额'],index = '省/自治区')

# df.groupby("省/自治区")['销售额'].mean() 通过上一章分组

### 3 - 数据透视｜指定方法

制作各省「销售总额」的数据透视表

In [ ]:
pd.pivot_table(df,values = ['销售额'],index = '省/自治区',aggfunc = sum)

### 4 - 数据透视｜多方法

制作各省「销售总额」与「平均销售额」的数据透视表

In [ ]:
pd.pivot_table(df,values = ['销售额'],index = '省/自治区',aggfunc = ['mean',sum])

### 5 - 数据透视｜多指标

制作各省市「销售总额」与「利润总额」的数据透视表

In [ ]:
pd.pivot_table(df,values = ['销售额','利润','数量'],index = '类别',aggfunc = sum)

### 6 - 数据透视｜多索引

制作「各省市」与「不同类别」产品「销售总额」的数据透视表

In [ ]:
pd.pivot_table(df,values = ['销售额'],index = ['省/自治区','类别'],aggfunc = sum)

### 7 - 数据透视｜多层

制作各省市「不同类别」产品的「销售总额」透视表

In [ ]:
pd.pivot_table(df,values = ['销售额'],index = ['省/自治区'], columns='类别',aggfunc = sum)

### 8 - 数据透视｜综合

制作「各省市」、「不同类别」产品「销售量与销售额」的「均值与总和」的数据透视表，并在最后追加一行『合计』

In [ ]:
pd.pivot_table(df,values = ['销售额','数量'],index = ['省/自治区','类别'],aggfunc = ['mean',sum],margins=True)

### 9 - 数据透视｜筛选

在上一题的基础上，查询 **「类别」** 等于 **「办公用品」** 的详情

In [81]:
table = pd.pivot_table(df,values = ['销售额','数量'],index = ['省/自治区','类别'],aggfunc = ['mean',sum],margins=True)
table.query('类别 == ["办公用品"]')

### 10 -数据透视｜逆透视

逆透视就是将宽的表转换为长的表，例如将第 5 题的透视表进行逆透视，其中不需要转换的列为『数量』列

In [ ]:
table = pd.pivot_table(df,values = ['销售额','利润','数量'],index = '类别',aggfunc = sum)
table.melt(id_vars=['数量'],var_name='分类',value_name='金额')

微信搜索公众号「早起Python」，关注后可以获得更多资源！`

## 数据合并

### concat - 数据拼接

`concat`主要用于**数据拼接**，也是非常常用的一个操作

除了官方文档外很难找到比官方文档更好的练习

以下案例来源或基于 [👉官方文档](https://pandas.pydata.org/pandas-docs/version/0.20/merging.html) 中的案例修改而来

在练习之前应执行下方代码生成数据

并应预览不同数据的结构以及每题的图解（如果有）

In [210]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])


df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])


df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])


df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df4

#### 11 - <font color = '#FB8E00'>concat</font>｜默认拼接

拼接 df1 和 df2

In [ ]:
pd.concat([df1, df2])

#### 12 - <font color = '#FB8E00'>concat</font>｜拼接多个

垂直拼接 `df1、df2、df3`，效果如下图所示

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_basic.png)

In [ ]:
pd.concat([df1, df2, df3])

#### 13 - <font color = '#FB8E00'>concat</font>｜重置索引

垂直拼接 df1 和 df4，并按顺序重新生成索引，

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_ignore_index.png)

In [ ]:
pd.concat([df1, df4], ignore_index=True)


#### 14 - <font color = '#FB8E00'>concat</font>｜横向拼接

横向拼接 `df1、df4`，效果如下图所示


![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_axis1.png)

In [ ]:
pd.concat([df1,df4],axis=1)

#### 15 - <font color = '#FB8E00'>concat</font>｜横向拼接（取交集）

在上一题的基础上，只取结果的交集

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_axis1_inner.png)

In [ ]:
pd.concat([df1,df4],axis=1,join='inner')

#### 16 - <font color = '#FB8E00'>concat</font>｜横向拼接（取指定）

在 14 题基础上，只取包含 df1 索引的部分

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_axis1_join_axes.png)

In [ ]:
pd.concat([df1, df4], axis=1).reindex(df1.index)

![公众号：早起Python](http://liuzaoqi.oss-cn-beijing.aliyuncs.com/2021/09/18/16319660121648.jpg?域名/sample.jpg?x-oss-process=style/stylename)

#### 17 - <font color = '#FB8E00'>concat</font>｜新增索引

拼接 `df1、df2、df3`，同时新增一个索引（`x、y、z`）来区分不同的表数据来源

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_concat_keys.png)

In [ ]:
pd.concat([df1, df2, df3], keys=['x', 'y', 'z'])

### merge - 数据连接

concat是拼接，merge则是连接，同样重要的一个操作

同样很难找到比官方文档更好的练习，以下案例来源或基于 [👉官方文档](https://pandas.pydata.org/pandas-docs/version/0.20/merging.html#database-style-dataframe-joining-merging) 中的案例修改而来

在练习之前应执行每题下方的代码生成数据

并应预览不同数据的结构以及每题的图解（如果有）

#### 18 - <font color = '#1B85FF' >merge</font>｜按单键

根据 `key` 连接 `left` 和 `right`

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key.png)

In [ ]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [4]:
pd.merge(left, right, on='key')

pd.merge(left, right, on='key')


#### 19 - <font color = '#1B85FF' >merge</font>｜按多键

根据 `key1` 和 `key2` 连接 `left` 和 `right`

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key_multiple.png)

In [235]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})


right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})


In [ ]:
pd.merge(left, right, on=['key1', 'key2'])

#### 20 - <font color = '#1B85FF' >merge</font>｜左外连接


如下图所示的结果连接 left 和 right，保留左表全部键
![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key_left.png)

In [ ]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

#### 21 - <font color = '#1B85FF' >merge</font>｜右外连接


如下图所示的结果连接 left 和 right，保留右表全部键
![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key_right.png)

In [ ]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

#### 22 - <font color = '#1B85FF' >merge</font>｜全外连接


如下图所示的结果连接 left 和 right，保留全部键
![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key_outer.png)

In [ ]:
pd.merge(left, right, how='outer', on=['key1', 'key2'])

#### 23 - <font color = '#1B85FF' >merge</font>｜内连接


如下图所示的结果连接 left 和 right，保留交集
![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_on_key_inner.png)

In [ ]:
pd.merge(left, right, how='inner', on=['key1', 'key2'])

#### 24 - <font color = '#1B85FF' >merge</font>｜重复索引


重新产生数据并按下图所示进行连接
![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_merge_overlapped_suffix.png)

In [255]:
left = pd.DataFrame({'k': ['K0', 'K1', 'K2'], 'v': [1, 2, 3]})

right = pd.DataFrame({'k': ['K0', 'K0', 'K3'], 'v': [4, 5, 6]})


In [ ]:
pd.merge(left, right, on='k', suffixes=['_l', '_r'])

### join - 组合

最后一个数据合并的常用且重要的操作是`join`

同样很难找到比官方文档更好的练习，以下案例来源或基于 [👉官方文档](https://pandas.pydata.org/pandas-docs/version/0.20/merging.html#database-style-dataframe-joining-merging) 中的案例修改而来

在练习之前应执行每题下方的代码生成数据

并应预览不同数据的结构以及每题的图解（如果有）

In [242]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                     index=['K0', 'K1', 'K2'])


right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                      index=['K0', 'K2', 'K3'])

#### 25 -  <font color ='#27BE49'>join</font>｜左对齐

合并 left 和 right，并按照 left 的索引进行对齐

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_join.png)

In [ ]:
left.join(right)

#### 26 - <font color ='#27BE49'>join</font>｜左对齐（外连接）

按下图所示进行连接

思考：merge 做法

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_join_outer.png)

In [ ]:
left.join(right, how='outer')

#### 27 - <font color ='#27BE49'>join</font>｜左对齐（内连接）

按下图所示进行连接

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_join_inner.png)

In [ ]:
left.join(right, how='inner')

#pd.merge(left, right, left_index=True, right_index=True, how='inner')

#### 28 - <font color ='#27BE49'>join</font>｜按索引

重新产生数据并按下图所示进行连接（根据 `key`）

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_join_key_columns.png)


In [248]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})


right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                      index=['K0', 'K1'])

In [ ]:
left.join(right, on='key')

#### 29 - <font color ='#27BE49'>join</font>｜按索引（多个）

重新产生数据并按下图所示进行连接（根据 `key1` 和 `key2`）

![](https://pandas.pydata.org/pandas-docs/version/0.20/_images/merging_join_multikeys.png)


In [ ]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})


index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                  ('K2', 'K0'), ('K2', 'K1')])


right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                   'D': ['D0', 'D1', 'D2', 'D3']},
                  index=index)

In [ ]:
left.join(right, on=['key1', 'key2'])

![](http://liuzaoqi.oss-cn-beijing.aliyuncs.com/2021/09/16/16317972442543.jpg?域名/sample.jpg?x-oss-process=style/stylename)